In [120]:
import copy
import numpy as np
import os
import pickle
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from PIL import Image
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate

# Load Data

In [142]:
person_pkl = open('./files/COCO/personimages.pkl', 'rb')
person_matrices = pickle.load(person_pkl)

no_person_pkl = open('./files/COCO/nopersonimages.pkl', 'rb')
no_person_matrices = pickle.load(no_person_pkl)

In [158]:
person_test_pkl = open('./files/COCO/personimagesTest.pkl', 'rb')
person_test_matrices = pickle.load(person_test_pkl)

no_person_test_pkl = open('./files/COCO/nopersonimagesTest.pkl', 'rb')
no_person_test_matrices = pickle.load(no_person_test_pkl)

In [210]:
class CocoDataset(Dataset):
    """ Our custom Coco Dataset """
    
    def __init__(self, matrix, person, transform=None):
        self.matrix = matrix
        self.transform = transform
        self.person = person
        
    def __len__(self):
        return len(self.matrix)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample = self.matrix[idx]
        person = self.person[idx]

        try:
            if self.transform:
                sample = (self.transform(Image.fromarray(sample[0])), person)
            else:
                sample = (Image.fromarray(sample[0]), person)
        except:
            sample = None

        return sample

In [211]:
num_samples = len(person_matrices)
dataset = CocoDataset(
    person_matrices + no_person_matrices,
    transform=torchvision.transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    person=np.concatenate((np.ones(num_samples, dtype=np.int_), np.zeros(num_samples, dtype=np.int_)))
)

In [212]:
num_test_samples = len(person_test_matrices)
test_dataset = CocoDataset(
    person_test_matrices + no_person_test_matrices,
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    person=np.concatenate((np.ones(num_test_samples, dtype=np.int_), np.zeros(num_test_samples, dtype=np.int_)))
)

In [213]:
dataset_sizes = {'train': num_samples*2, 'val': num_test_samples*2}

In [214]:
def collate_fn_v(batch):
    print(batch)
    batch = list(filter(lambda x : x is not None, batch))
    return default_collate(batch)

In [215]:
train_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

In [216]:
dataloaders = {'train': train_loader, 'val': test_loader}

In [217]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Modeling

In [218]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs - 1))
        print("-" * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [219]:
resnet = models.resnet50(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = False
resnet.fc = nn.Linear(2048, 2)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [220]:
resnet = train_model(resnet, criterion, optimizer_ft, exp_lr_scheduler, dataloaders, dataset_sizes, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.8490 Acc: 0.5500
val Loss: 0.5073 Acc: 0.6875

Epoch 1/24
----------
train Loss: 1.0097 Acc: 0.5750
val Loss: 0.6041 Acc: 0.6875

Epoch 2/24
----------
train Loss: 0.8588 Acc: 0.6375
val Loss: 0.5727 Acc: 0.7250

Epoch 3/24
----------
train Loss: 0.8189 Acc: 0.6667
val Loss: 0.5114 Acc: 0.7625

Epoch 4/24
----------
train Loss: 0.7190 Acc: 0.6792
val Loss: 0.5492 Acc: 0.7750

Epoch 5/24
----------
train Loss: 0.9461 Acc: 0.6708
val Loss: 0.5754 Acc: 0.7375

Epoch 6/24
----------
train Loss: 1.0097 Acc: 0.6417
val Loss: 1.7395 Acc: 0.6125

Epoch 7/24
----------
train Loss: 0.8003 Acc: 0.6833
val Loss: 0.5324 Acc: 0.7625

Epoch 8/24
----------
train Loss: 0.5800 Acc: 0.7083
val Loss: 0.6160 Acc: 0.7125

Epoch 9/24
----------
train Loss: 0.5594 Acc: 0.7458
val Loss: 0.6140 Acc: 0.7375

Epoch 10/24
----------
train Loss: 0.5697 Acc: 0.7292
val Loss: 0.5755 Acc: 0.7625

Epoch 11/24
----------
train Loss: 0.5365 Acc: 0.7417
val Loss: 0.5674 Acc: 0.7375

Ep